In [ ]:
import subprocess
import sys

# List of packages to install
packages = ["h3", "python-geohash", "geopy", "seaborn", "geopandas"]

# Function to install a package
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Try to import packages, install if not found
try:
    import h3
    import geohash
    import pandas as pd
    import geopy.geocoders
    import geopandas as gpd
    from shapely import wkt
    import matplotlib.pyplot as plt
    from shapely.geometry import LineString, Polygon
except ImportError as e:
    missing_package = str(e).split("'")[1]
    print(f"Package {missing_package} not found. Installing...")
    if missing_package == "h3":
        install_package("h3==4.0.0b5")  # replace with the pre-release version if necessary
    elif missing_package == "geohash":
        install_package("python-geohash")
    else:
        install_package(missing_package)
    
    # Try importing again
    try:
        import h3
        import geohash
        import pandas as pd
        import geopy.geocoders
        import geopandas as gpd
        from shapely import wkt
        import matplotlib.pyplot as plt
        from shapely.geometry import LineString, Polygon
    except ImportError as e:
        print(f"Failed to install {missing_package}. Please try to install it manually.")

print("All packages are successfully imported!")

In [ ]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [ ]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# dl

## Get example dataset

In [ ]:
country_code = 'MX'
start_date = 20191201
end_date = 201901210

# Read data from SQL table
query = f"""
    SELECT 
        -- *,
        
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        timezoneoffset_secs,
        lat,
        lng, 
        
        -- Extract only the date part
        date(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
        interval '1' second * timezoneoffset_secs) AS event_date_utc
        
        -- Extract the date and time part and adjust by the timezone offset 
        -- date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
        -- interval '1' second * timezoneoffset_secs AS event_datetime_utc
        
    FROM {pe_dl_table}
    WHERE 
        processing_date = {start_date}
        AND country_code = '{country_code}'
        AND event_zoned_datetime IS NOT NULL
"""        
  

pe_dl_table_df = sql_engine.read_sql(query)
pe_dl_table_df

In [ ]:
pe_dl_table_df.to_csv(f'/home/jovyan/Data/20191201_{country_code}_dl.csv', index=False) 

## Read in 

In [ ]:
df = pd.read_csv(f'/home/jovyan/Data/20191201_CO_dl.csv')
df

## Convert the points to H3 hexagons

In [ ]:
resolution = 7
df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row['lat'], row['lng'], resolution), axis=1)
df

In [ ]:
# Group by H3 index to get the number of points and unique users
h3_agg = df.groupby('h3_index').agg(
    points_count=('cuebiq_id', 'count'),
    users_count=('cuebiq_id', 'nunique')
).reset_index()
h3_agg

In [ ]:
# Filter rows where users_count > 10
h3_agg = h3_agg[h3_agg['users_count'] > 10]

# Display the filtered DataFrame
h3_agg

## Histogram

In [ ]:
# Check the maximum, minimum, mean, and median values for points_count
max_points = h3_agg['points_count'].max()
min_points = h3_agg['points_count'].min()
mean_points = h3_agg['points_count'].mean()
median_points = h3_agg['points_count'].median()

# Check the maximum, minimum, mean, and median values for users_count
max_users = h3_agg['users_count'].max()
min_users = h3_agg['users_count'].min()
mean_users = h3_agg['users_count'].mean()
median_users = h3_agg['users_count'].median()

print(f"Max Points Count: {max_points}, Min Points Count: {min_points}, Mean Points Count: {mean_points:.2f}, Median Points Count: {median_points}")
print(f"Max Users Count: {max_users}, Min Users Count: {min_users}, Mean Users Count: {mean_users:.2f}, Median Users Count: {median_users}")

### Points Count 

country_codes = ['MX', 'IN', 'ID', 'CO']   
colors = ['#5c6c84', '#83a0b2', '#afc8bb', '#d9ccbc'] 

In [ ]:
# Plot histogram for points_count
plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(h3_agg['points_count'], bins=30, density=False, color='#d9ccbc', edgecolor='black', alpha=0.7)

# Fit a normal distribution to the data
mu, std = norm.fit(h3_agg['points_count'])

# Plot the PDF
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'N({mu:.2f}, {std:.2f})')

plt.title('Distribution of Points Count per H3 Index')
plt.xlabel('Points Count')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# zoom in to  histogram Plot of points_count
plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(h3_agg['points_count'], bins= 1000, density=False, color='#d9ccbc', edgecolor='black', alpha=0.7)

# Fit a normal distribution to the data
# mu, std = norm.fit(h3_agg['points_count'])

# # Plot the PDF
xmin, xmax = 0, 1000  # Set the x-axis limits to zoom in
# x = np.linspace(xmin, xmax, 100)
# p = norm.pdf(x, mu, std)
# plt.plot(x, p, 'k', linewidth=2, label=f'N({mu:.2f}, {std:.2f})')

plt.xlim(xmin, xmax) # Set x-axis limits

plt.title('Distribution of Points Count per H3 Index')
plt.xlabel('Points Count')
plt.ylabel('Density')
# plt.legend()
plt.grid(True)
plt.show()


### Plot histogram for users_count

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(h3_agg['users_count'], bins=50, color='#d9ccbc', edgecolor='black')
plt.title('Distribution of Users Count per H3 Index')
plt.xlabel('Users Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# zoom in to  histogram Plot of points_count
plt.figure(figsize=(10, 6))
n, bins, patches = plt.hist(h3_agg['users_count'], bins= 1000, density=False, color='#d9ccbc', edgecolor='black', alpha=0.7)

# Fit a normal distribution to the data
mu, std = norm.fit(h3_agg['users_count'])

# Plot the PDF
xmin, xmax = 5, 100  # Set the x-axis limits to zoom in
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2, label=f'N({mu:.2f}, {std:.2f})')

# Set x-axis limits
plt.xlim(xmin, xmax)

plt.title('Distribution of User Count per H3 Index')
plt.xlabel('Points Count')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


## Map (h3_index to POLYGON)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

def h3_to_polygon(h3_index):
    boundary = h3.cell_to_boundary(h3_index)
    return Polygon([(lat, lon) for lat, lon in boundary])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(h3_agg, geometry=h3_agg['h3_index'].apply(h3_to_polygon))
gdf


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h3
from scipy.stats import norm

# Define the countries and corresponding colors
country_codes = ['MX', 'IN', 'ID', 'CO']
colors = ['#5c6c84', '#83a0b2', '#afc8bb', '#d9ccbc']

fig, axes = plt.subplots(4, 2, figsize=(15, 20))

for i, (country_code, color) in enumerate(zip(country_codes, colors)):
    # Load data for each country
    df = pd.read_csv(f'/home/jovyan/Data/20191201_{country_code}_dl.csv')

    # Calculate H3 index
    resolution = 7
    df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row['lat'], row['lng'], resolution), axis=1)

    # Group by H3 index to get the number of points and unique users
    h3_agg = df.groupby('h3_index').agg(
        points_count=('cuebiq_id', 'count'),
        users_count=('cuebiq_id', 'nunique')
    ).reset_index()

    # Filter rows where users_count > 10
    h3_agg = h3_agg[h3_agg['users_count'] > 10]

    # Plot histogram for points_count
    ax = axes[i, 0]
    n, bins, patches = ax.hist(h3_agg['points_count'], bins=30, density=False, color=color, edgecolor='black', alpha=0.7)

    # Fit a normal distribution to the data
    mu, std = norm.fit(h3_agg['points_count'])

    # Plot the PDF
    xmin, xmax = ax.get_xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    ax.plot(x, p, 'k', linewidth=2, label=f'N({mu:.2f}, {std:.2f})')

    ax.set_title(f'{country_code} - Distribution of Points Count per H3 Index')
    ax.set_xlabel('Points Count')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True)

    # Plot histogram for users_count
    ax = axes[i, 1]
    ax.hist(h3_agg['users_count'], bins=30, color=color, edgecolor='black', alpha=0.7)
    ax.set_title(f'{country_code} - Distribution of Users Count per H3 Index')
    ax.set_xlabel('Users Count')
    ax.set_ylabel('Frequency')
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h3
from scipy.stats import norm

# Define the countries and corresponding colors
country_codes = ['MX', 'IN', 'ID', 'CO']
colors = ['#5c6c84', '#83a0b2', '#afc8bb', '#d9ccbc']

# Define the x-axis limits for points_count and users_count
xmax_points = [2500, 2500, 2500, 2500]
xmax_users = [300, 300, 300, 300]

fig, axes = plt.subplots(4, 2, figsize=(15, 20))

for i, (country_code, color) in enumerate(zip(country_codes, colors)):
    # Load data for each country
    df = pd.read_csv(f'/home/jovyan/Data/20191201_{country_code}_dl.csv')

    # Calculate H3 index
    resolution = 7
    df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row['lat'], row['lng'], resolution), axis=1)

    # Group by H3 index to get the number of points and unique users
    h3_agg = df.groupby('h3_index').agg(
        points_count=('cuebiq_id', 'count'),
        users_count=('cuebiq_id', 'nunique')
    ).reset_index()

    # Filter rows where users_count > 10
    h3_agg = h3_agg[h3_agg['users_count'] > 10]

    # Plot histogram for points_count
    ax = axes[i, 0]
    n, bins, patches = ax.hist(h3_agg['points_count'], bins=500, density=False, color=color, edgecolor='black', alpha=0.7)
    
    xmin, xmax = 0, xmax_points[i]  # Set the x-axis limits to zoom in
    ax.set_xlim(xmin, xmax)
    ax.set_title(f'{country_code} - Distribution of Points Count per H3 Index')
    ax.set_xlabel('Points Count')
    ax.set_ylabel('Frequency')
    ax.grid(True)

    # Plot histogram for users_count
    ax = axes[i, 1]
    ax.hist(h3_agg['users_count'], bins=500, color=color, edgecolor='black', alpha=0.7)
    ax.set_xlim(0, xmax_users[i])  # Set the x-axis limits to zoom in
    ax.set_title(f'{country_code} - Distribution of Users Count per H3 Index')
    ax.set_xlabel('Users Count')
    ax.set_ylabel('Frequency')
    ax.grid(True)

plt.tight_layout()
plt.savefig(f'/home/jovyan/Data/20191201_dl_plots_1.png', dpi=300)  # Save the figure with high resolution
plt.show()


# Tj

In [ ]:
df['trajectory_wkt'][0]

In [ ]:
# Parse the WKT strings into geometries
df['geometry'] = df['trajectory_wkt'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(ax=ax, color='blue')

# Customize the plot
plt.title('Trajectory Data')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()